In [ ]:
!pip install category_encoders

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from category_encoders import OrdinalEncoder
from category_encoders import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier 
from sklearn.compose import make_column_selector
from sklearn.compose import make_column_transformer
from sklearn.inspection import permutation_importance
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from yellowbrick.model_selection import feature_importances
from yellowbrick.features import pca_decomposition
from yellowbrick.target import class_balance
from yellowbrick.target.feature_correlation import feature_correlation
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
df = pd.read_csv('Measurement_summary.csv')
df.info()

In [ ]:
df.head(5)

In [ ]:
df.isnull().sum()

In [ ]:
df.dropna(inplace=True)
df.isnull().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
df.nunique()

In [ ]:
df_tt = df.copy()
df_tt['Measurement date'] = df_tt['Measurement date'].astype('datetime64')
df_tt['hour'] = df_tt.loc[:, "Measurement date"].dt.hour
df_tt = df_tt.drop('Measurement date', axis=1)

time = df_tt.groupby('hour', as_index=False).agg({'PM10':'mean', 'PM2.5':'mean'})
time.plot(x='hour', y=['PM10', 'PM2.5'])

In [ ]:
df_y = df.copy()
df_y['Measurement date'] = df_y['Measurement date'].astype('datetime64')
df_y['Year'] = df_y.loc[:, "Measurement date"].dt.year
df_y = df_y.drop('Measurement date', axis=1)

year = df_y.groupby('Year', as_index=False).agg({'SO2':'mean', 'NO2':'mean', 'O3':'mean', 'CO':'mean', 'PM10':'mean', 'PM2.5':'mean'})
plt.plot(year.Year, year['PM10'], label='PM10')
plt.plot(year.Year, year['PM2.5'], label='PM2.5')
plt.xticks([2017,2018,2019])
plt.legend()

In [ ]:
df_a = df.copy()
df_a = df_a.groupby(['Measurement date'], as_index=False).agg({'SO2':'mean', 'NO2':'mean', 'O3':'mean', 'CO':'mean', 'PM10':'mean', 'PM2.5':'mean'})
df_air = df_a.corr()
sns.clustermap(df_air, annot=True, cmap='Reds', vmin=-1, vmax=1)

In [ ]:
# 서울 미세먼지(PM10) 농도
location = df.groupby('Station code')['PM10'].agg([np.mean])
location['Latitude'] = df['Latitude'].unique()
location['Longitude'] = df['Longitude'].unique()

import folium
from folium.plugins import MarkerCluster

# PM10
def PM10(x):
    if x >= 45:
        return 'red'
    elif x >= 40:
        return 'yellow'
    else:
        return 'cyan'

# Map
seoul = folium.Map(location=[37.55138077230307, 126.98712254969668], zoom_start=12)

# Circle
for i in range(len(location)):
    # 관측소
    folium.Circle(location=[location.iloc[i,1], location.iloc[i,2]], radius = location.iloc[i, 0]*30, color=PM10(location.iloc[i,0]),fill_color='#ffffgg').add_to(seoul)
seoul

In [ ]:
df_t = df.groupby(['Measurement date'], as_index=False).agg({'SO2':'mean', 'NO2':'mean', 'O3':'mean', 'CO':'mean', 'PM10':'mean', 'PM2.5':'mean'})
df_t

In [ ]:
f, ax = plt.subplots(4, 2, figsize=(20,15))

sns.scatterplot(x='NO2', y= 'PM10', data=df_t, ax=ax[0,0], alpha=.5)
sns.scatterplot(x='NO2', y= 'PM2.5', data=df_t, ax=ax[0,1], alpha=.5)

sns.scatterplot(x='CO', y= 'PM10', data=df_t, ax=ax[1,0], alpha=.5)
sns.scatterplot(x='CO', y= 'PM2.5', data=df_t, ax=ax[1,1], alpha=.5)

sns.scatterplot(x='SO2', y= 'PM10', data=df_t, ax=ax[2,0], alpha=.5)
sns.scatterplot(x='SO2', y= 'PM2.5', data=df_t, ax=ax[2,1], alpha=.5)

sns.scatterplot(x='O3', y= 'PM10', data=df_t, ax=ax[3,0], alpha=.5)
sns.scatterplot(x='O3', y= 'PM2.5', data=df_t, ax=ax[3,1], alpha=.5)
plt.plot()

In [ ]:
sns.scatterplot(x='NO2', y= 'PM10', data=df_t, label='NO2')
sns.scatterplot(x='O3', y= 'PM10', data=df_t, label='O3')
plt.plot()

In [ ]:
sns.scatterplot(x='NO2', y= 'PM2.5', data=df_t, label='NO2')
sns.scatterplot(x='O3', y= 'PM2.5', data=df_t, label='O3')
plt.plot()

In [ ]:
# df_time = df['Measurement date'].str.split(" ",n=1,expand=True)
# df['date'] = df_time[0]
# df['time'] = df_time[1]
# df = df.drop(['Measurement date'],axis = 1)
# df_add = df.loc[:,['SO2','NO2','O3','CO','PM10','PM2.5','time']]

# plt.figure(figsize = (15,13))
# plt.bar(df_add['time'],df_add['PM10'],color = 'aquamarine')
# plt.title('Concentration of fine dust by time',fontsize = 20)
# plt.xlabel('Time',fontsize=15)
# plt.ylabel('Concentration',fontsize = 15)
# plt.show()

In [ ]:
dff = df.groupby(['Measurement date'], as_index=False).agg({'SO2':'mean', 'NO2':'mean', 'O3':'mean', 'CO':'mean', 'PM10':'mean', 'PM2.5':'mean'})

In [ ]:
## PM2.5 Sub-Index calculation
def get_PM25_subindex(x):
    if x <= 30:
        return x * 50 / 30
    elif x <= 60:
        return 50 + (x - 30) * 50 / 30
    elif x <= 90:
        return 100 + (x - 60) * 100 / 30
    elif x <= 120:
        return 200 + (x - 90) * 100 / 30
    elif x <= 250:
        return 300 + (x - 120) * 100 / 130
    elif x > 250:
        return 400 + (x - 250) * 100 / 130
    else:
        return 0

dff["PM2.5_SubIndex"] = dff["PM2.5"].apply(lambda x: get_PM25_subindex(x))

## PM10 Sub-Index calculation
def get_PM10_subindex(x):
    if x <= 50:
        return x
    elif x <= 100:
        return x
    elif x <= 250:
        return 100 + (x - 100) * 100 / 150
    elif x <= 350:
        return 200 + (x - 250)
    elif x <= 430:
        return 300 + (x - 350) * 100 / 80
    elif x > 430:
        return 400 + (x - 430) * 100 / 80
    else:
        return 0

dff["PM10_SubIndex"] = dff["PM10"].apply(lambda x: get_PM10_subindex(x))

## SO2 Sub-Index calculation
def get_SO2_subindex(x):
    if x <= 40:
        return x * 50 / 40
    elif x <= 80:
        return 50 + (x - 40) * 50 / 40
    elif x <= 380:
        return 100 + (x - 80) * 100 / 300
    elif x <= 800:
        return 200 + (x - 380) * 100 / 420
    elif x <= 1600:
        return 300 + (x - 800) * 100 / 800
    elif x > 1600:
        return 400 + (x - 1600) * 100 / 800
    else:
        return 0

dff["SO2_SubIndex"] = dff["SO2"].apply(lambda x: get_SO2_subindex(x))

## NO2 Sub-Index calculation
def get_NO2_subindex(x):
    if x <= 40:
        return x * 50 / 40
    elif x <= 80:
        return 50 + (x - 40) * 50 / 40
    elif x <= 180:
        return 100 + (x - 80) * 100 / 100
    elif x <= 280:
        return 200 + (x - 180) * 100 / 100
    elif x <= 400:
        return 300 + (x - 280) * 100 / 120
    elif x > 400:
        return 400 + (x - 400) * 100 / 120
    else:
        return 0

dff["NO2_SubIndex"] = dff["NO2"].apply(lambda x: get_NO2_subindex(x))

## CO Sub-Index calculation
def get_CO_subindex(x):
    if x <= 1:
        return x * 50 / 1
    elif x <= 2:
        return 50 + (x - 1) * 50 / 1
    elif x <= 10:
        return 100 + (x - 2) * 100 / 8
    elif x <= 17:
        return 200 + (x - 10) * 100 / 7
    elif x <= 34:
        return 300 + (x - 17) * 100 / 17
    elif x > 34:
        return 400 + (x - 34) * 100 / 17
    else:
        return 0

dff["CO_SubIndex"] = dff["CO"].apply(lambda x: get_CO_subindex(x))

## O3 Sub-Index calculation
def get_O3_subindex(x):
    if x <= 50:
        return x * 50 / 50
    elif x <= 100:
        return 50 + (x - 50) * 50 / 50
    elif x <= 168:
        return 100 + (x - 100) * 100 / 68
    elif x <= 208:
        return 200 + (x - 168) * 100 / 40
    elif x <= 748:
        return 300 + (x - 208) * 100 / 539
    elif x > 748:
        return 400 + (x - 400) * 100 / 539
    else:
        return 0

dff["O3_SubIndex"] = dff["O3"].apply(lambda x: get_O3_subindex(x))

## AQI bucketing
def get_AQI_bucket(x):
    if x <= 50:
        return "Good"
    elif x <= 100:
        return "Satisfactory"
    elif x <= 200:
        return "Moderate"
    elif x <= 300:
        return "Bad"
    elif x <= 400:
        return "Very Bad"
    elif x > 400:
        return "Hell"
    else:
        return np.NaN

dff["AQI_score"] = round(dff[["PM2.5_SubIndex","PM10_SubIndex","SO2_SubIndex","NO2_SubIndex","CO_SubIndex","O3_SubIndex"]].max(axis = 1))

dff["AQI_warn"] = dff["AQI_score"].apply(lambda x: get_AQI_bucket(x))
# dff.drop(["PM2.5_SubIndex","PM10_SubIndex","SO2_SubIndex","NO2_SubIndex","CO_SubIndex","O3_SubIndex"], axis=1, inplace=True)
dff.drop(["PM2.5","PM10","SO2","NO2","CO","O3"], axis=1, inplace=True)
dff.head(13)

In [ ]:
a = pd.DataFrame(dff.AQI_warn.value_counts())
b = pd.DataFrame(dff.AQI_warn.value_counts(normalize=True))
a.rename(columns={'AQI_warn':'Counts'},inplace=True)
b.rename(columns={'AQI_warn':'proportion'},inplace=True)
c = pd.concat([a, b], axis = 1)
c

In [ ]:
## 훈련/검증/테스트 split
train, test = train_test_split(dff, test_size=0.2, random_state=2)
train, val = train_test_split(train, test_size=len(test), random_state=2)
train.shape, val.shape, test.shape

In [ ]:
target = 'AQI_warn'

features = dff.columns.drop([target,
                             'Measurement date',
                             'AQI_score',
                             'PM10_SubIndex',
                             'PM2.5_SubIndex'
                             ])
X_train = train[features]
y_train = train[target]

X_val = val[features]
y_val = val[target]

X_test = test[features]
y_test = test[target]

In [ ]:
def fit_Dtrees(X_train, y_train):
    pipe = None    
    clf = None     

    pipe = make_pipeline(
        SimpleImputer(),
        StandardScaler(), 
        DecisionTreeClassifier(random_state=42, criterion='entropy')
    )
    params = {
        "decisiontreeclassifier__max_depth" : [3,4,5,6,7],
        "decisiontreeclassifier__min_samples_split" :[2,3]
    }
    
    clf = RandomizedSearchCV(
        pipe, 
        param_distributions=params, 
        n_iter=10, 
        cv=5,
        scoring='accuracy',
        n_jobs=-1
    )
    clf.fit(X_train, y_train)

    return clf

In [ ]:
clf_D = fit_Dtrees(X_train, y_train)

dt = clf_D.best_estimator_

pred_train = dt.predict(X_train)
pred_val = dt.predict(X_val) 
pred_test = dt.predict(X_test) 

print('Train 정확도:', accuracy_score(y_train, pred_train))
print('val 정확도:', accuracy_score(y_val, pred_val))
print('Test 정확도:', accuracy_score(y_test, pred_test))

In [ ]:
model_dt = dt.named_steps['decisiontreeclassifier']

importances = pd.Series(model_dt.feature_importances_, X_val.columns)
plt.figure(figsize=(5,5))
importances.sort_values().plot.barh();

In [ ]:
max_depth_list = range(1,20)

train_acc_list = []
val_acc_list = []
for max_depth in max_depth_list:
    tree = DecisionTreeClassifier(max_depth=max_depth, random_state=2)
    tree.fit(X_train, y_train)

    pred_train = tree.predict(X_train)
    pred_val = tree.predict(X_val)

    train_acc_list.append(accuracy_score(y_train, pred_train))
    val_acc_list.append(accuracy_score(y_val, pred_val))

import pandas as pd
d = {
    "max_depth":max_depth_list,
    "Train_acc":train_acc_list,
    "Val_acc":val_acc_list
}
acc_df = pd.DataFrame(d)
acc_df.set_index('max_depth').plot(figsize=(10,7))
plt.xticks(max_depth_list)
plt.ylabel('Accuracy')
plt.show()


In [ ]:
fig, ax = plt.subplots()
pcm = plot_confusion_matrix(dt, X_val, y_val,
                            cmap=plt.cm.Blues,
                            ax=ax)
plt.title(f'Confusion matrix, n = {len(y_val)}', fontsize=15)
plt.show()

In [ ]:
y_pred_proba = dt.predict_proba(X_val)[:, 1]
pred_proba = pd.DataFrame({
    'y_val': y_val,
    'pred_proba': y_pred_proba})

from sklearn.metrics import roc_curve

label=list(dff.AQI_warn.unique())
for i in label:
  fpr, tpr, thresholds = roc_curve(y_val, y_pred_proba, pos_label=i)

  roc = pd.DataFrame({
      'FPR(Fall-out)': fpr, 
      'TPRate(Recall)': tpr, 
      'Threshold': thresholds
  })
  plt.scatter(fpr, tpr, label=i)
  plt.legend()
  plt.title('ROC curve')
  plt.xlabel('FPR(Fall-out)')
  plt.ylabel('TPR(Recall)');


In [ ]:
randf= make_pipeline(
    SimpleImputer(),
    RandomForestClassifier(random_state=10)
)
randf.fit(X_train, y_train)
pred = randf.predict(X_val)
print('RandomForest 정확도(튜닝X):', accuracy_score(y_val, pred))

In [ ]:
rf_clf = RandomForestClassifier(random_state=10)
params = { 
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}
grid_cv = GridSearchCV(rf_clf , param_grid=params , cv=2 ,verbose=1)
grid_cv.fit(X_train , y_train)
print('최적 하이퍼 파라미터:\n', grid_cv.best_params_)
print('최고 예측 정확도:', grid_cv.best_score_)

rf = grid_cv.best_estimator_
pred = rf.predict(X_val)
print('RandomForest 정확도(튜닝O)', accuracy_score(y_val, pred))

In [ ]:
fig, ax = plt.subplots()
pcm = plot_confusion_matrix(randf, X_test, y_test,
                            cmap=plt.cm.Blues,
                            ax=ax)
plt.title(f'Confusion matrix, n = {len(y_test)}', fontsize=15)
plt.show()

In [ ]:
model2 = make_pipeline(
    SimpleImputer(),
    GradientBoostingClassifier(random_state=10)
)
model2.fit(X_train, y_train)
pred = model2.predict(X_val)
print('GBM 정확도(튜닝X):', accuracy_score(y_val, pred))

In [ ]:
gb_clf = GradientBoostingClassifier(random_state=10)
params = {
        "learning_rate" : [0.05,0.1],
        "n_estimators" :[1000,1500]
    }
grid_cv = GridSearchCV(gb_clf , param_grid=params , cv=2 ,verbose=1)
grid_cv.fit(X_train , y_train)
print('최적 하이퍼 파라미터:\n', grid_cv.best_params_)
print('최고 예측 정확도:', grid_cv.best_score_)

gbm = grid_cv.best_estimator_
pred = gbm.predict(X_val)
print('GBM 정확도(튜닝O)', accuracy_score(y_val, pred))

In [ ]:
fig, ax = plt.subplots()
pcm = plot_confusion_matrix(gbm, X_val, y_val,
                            cmap=plt.cm.Blues,
                            ax=ax)
plt.title(f'Confusion matrix, n = {len(y_val)}', fontsize=15)
plt.show()

In [ ]:
xgbc = make_pipeline(
    SimpleImputer(),
    XGBClassifier(random_state=10)
)
xgbc.fit(X_train, y_train)
pred = xgbc.predict(X_val)
print('XGB 정확도(튜닝X):', accuracy_score(y_val, pred))

In [ ]:
xgb_clf = XGBClassifier(random_state=10)
params = {
        "learning_rate" : [0.05,0.1],
        "n_estimators" :[1000,1500]
    }
grid_cv = GridSearchCV(xgb_clf , param_grid=params , cv=2 ,verbose=1)
grid_cv.fit(X_train , y_train)
print('최적 하이퍼 파라미터:\n', grid_cv.best_params_)
print('최고 예측 정확도:', grid_cv.best_score_)

xgb = grid_cv.best_estimator_
pred = xgb.predict(X_val)
print('XGB 정확도(튜닝O)', accuracy_score(y_val, pred))

In [ ]:
fig, ax = plt.subplots()
pcm = plot_confusion_matrix(xgbc, X_val, y_val,
                            cmap=plt.cm.Blues,
                            ax=ax)
plt.title(f'Confusion matrix, n = {len(y_val)}', fontsize=15)
plt.show()

In [ ]:
def test(model):
  
  pred = model.predict(X_test)
  
  print('정확도 :', accuracy_score(y_test, pred))
  print('정밀도 :', precision_score(y_test, pred, average='weighted'))
  print('재현율 :', recall_score(y_test, pred, average='weighted'))
  print('f1 score :', f1_score(y_test, pred, average='weighted'))


In [ ]:
test(dt)

In [ ]:
test(randf)

In [ ]:
test(gbm)

In [ ]:
test(xgbc)